In [3]:
import os
from chunker import PDFChunker
from vectorizer import TextVectorizer
from retriever import Retriever
from generator import Generator
from tqdm import tqdm
import time
import numpy as np
from mistralai import Mistral
import fitz
import re

# Explore PDF

In [178]:
doc = fitz.open("../data/PDM_Porto_Aviso n.º 1934_2023.pdf")
metadata = doc.metadata
text = ""
for page in doc:
    text += page.get_text("text")  # Options: "text", "html", "xml"

In [184]:
text = text.split('Republicação do Regulamento do Plano Diretor Municipal do Porto')[1]

In [279]:
import re

# Split the text by the pattern "Artigo" followed by some numbering
pattern = r"Artigo[\s\S]*?(?=TÍTULO|CAPÍTULO|SECÇÃO|SUBSECÇÃO|Artigo|ANEXOS)"
matches = re.findall(pattern, text)

In [280]:
clean_matches = []

for match in matches:
    # The regex pattern to match the target text
    pattern = r"N\.º 20\s+27 de janeiro de 2023\s+Pág\. .*?\s+Diário da República, 2\.\ª série\s+PARTE .*"

    # Remove the matched pattern
    clean_match = re.sub(pattern, '', match)
    clean_matches.append(clean_match)

# Chunker

In [4]:
chunker = PDFChunker('../data/PDM_Porto_Aviso n.º 1934_2023.pdf')

In [5]:
chunks = chunker.split_pdf_by_articles()

In [6]:
len(chunks)

166

In [7]:
[len(chunk.split()) for chunk in chunks]

[232,
 234,
 805,
 182,
 431,
 466,
 240,
 63,
 29,
 47,
 141,
 61,
 64,
 220,
 144,
 420,
 66,
 66,
 102,
 129,
 65,
 74,
 128,
 450,
 57,
 80,
 533,
 228,
 49,
 284,
 62,
 172,
 33,
 27,
 70,
 44,
 63,
 44,
 46,
 173,
 150,
 93,
 75,
 82,
 51,
 29,
 169,
 33,
 30,
 56,
 135,
 62,
 18,
 54,
 29,
 28,
 129,
 108,
 118,
 106,
 106,
 230,
 98,
 140,
 77,
 125,
 151,
 149,
 102,
 229,
 254,
 119,
 162,
 54,
 129,
 257,
 258,
 2606,
 201,
 70,
 55,
 86,
 58,
 42,
 178,
 129,
 84,
 207,
 97,
 146,
 83,
 123,
 346,
 45,
 57,
 29,
 29,
 120,
 59,
 109,
 85,
 99,
 67,
 144,
 264,
 89,
 61,
 210,
 84,
 180,
 130,
 149,
 199,
 232,
 358,
 114,
 178,
 188,
 79,
 205,
 907,
 389,
 190,
 86,
 226,
 134,
 61,
 239,
 140,
 91,
 161,
 114,
 113,
 134,
 71,
 329,
 263,
 46,
 115,
 84,
 237,
 123,
 133,
 181,
 74,
 75,
 95,
 94,
 82,
 79,
 101,
 183,
 96,
 74,
 231,
 108,
 128,
 284,
 160,
 194,
 171,
 3257,
 53,
 78,
 42,
 26]

In [8]:
for i, chunk in enumerate(chunks):
    if len(chunk.split()) > 4096:
        print(i)

In [9]:
chunker.save_chunks_to_file(chunks, '../data/artigos.txt')

Chunks saved to ../data/artigos.txt


In [17]:
chunks[2]

'Artigo 3.º\nDefinições\nPara além dos conceitos definidos na legislação e regulamentos aplicáveis, para efeitos do \npresente Regulamento são adotados os seguintes conceitos técnicos:\na) Alinhamento dominante — o alinhamento dos edifícios ou vedações com maior extensão \nnuma dada frente urbana;\nb) Alinhamento dominante de tardoz — o alinhamento das fachadas de tardoz dos corpos \ndominantes dos edifícios que constituem a frente urbana, com maior extensão;\n\nc) Altura dominante das fachadas — é a que apresenta maior extensão ao longo do alinhamento \nurbano entre arruamentos concorrentes;\nd) Área de edificação, também designada ae — o somatório da área de cada um dos pisos, \nexpresso em metros quadrados (m2), de todos os edifícios que existem ou podem ser realizados \nna(s) parcela(s), com exclusão de:\ni) Terraços descobertos, varandas, desde que não envidraçadas, e balcões abertos para o \nexterior;\nii) Espaços livres de uso público cobertos pelas edificações;\niii) Sótão sem 

## Enrich chunks with hierarchy information

In [20]:
chunks = [chunk.replace("\n\n", "\n") for chunk in chunks]
chunker.save_chunks_to_file(chunks, '../data/artigos.txt')

Chunks saved to ../data/artigos.txt


In [21]:
article_hierarchy = {
    "1-6": "TITULO I - DISPOSIÇÕES GERAIS",
    "7-8": "TITULO II - SERVIDÕES ADMINISTRATIVAS E RESTRIÇÕES DE UTILIDADE PÚBLICA",
    "9-11": "TITULO III - USO DO SOLO - CAPÍTULO I - CLASSIFICAÇÃO E QUALIFICAÇÃO DO SOLO",
    "12-13": "TITULO III - USO DO SOLO - CAPÍTULO II - QUALIFICAÇÃO OPERATIVA",
    "14-16": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO I - DISPOSIÇÕES GERAIS",
    "17-18": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO II - ESPAÇOS CENTRAIS",
    "19-22": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO II - ESPAÇOS CENTRAIS - SUBSECÇÃO I - ÁREA HISTÓRICA",
    "23-25": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO II - ESPAÇOS CENTRAIS - SUBSECÇÃO II - ÁREA DE FRENTE URBANA CONTÍNUA DE TIPO I",
    "26-28": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO II - ESPAÇOS CENTRAIS - SUBSECÇÃO III - ÁREA DE FRENTE URBANA CONTÍNUA DE TIPO II",
    "29-30": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO II - ESPAÇOS CENTRAIS - SUBSECÇÃO IV - ÁREA DE EDIFÍCIOS DE TIPO MORADIA",
    "31-33": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO II - ESPAÇOS CENTRAIS - SUBSECÇÃO V - ÁREA DE BLOCOS ISOLADOS DE IMPLANTAÇÃO LIVRE",
    "34-34": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO III - ESPAÇOS DE ATIVIDADES ECONÓMICAS",
    "35-36": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO III - ESPAÇOS DE ATIVIDADES ECONÓMICAS - SUBSECÇÃO I - ÁREAS DE ATIVIDADES ECONÓMICAS DE TIPO I",
    "37-38": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO III - ESPAÇOS DE ATIVIDADES ECONÓMICAS - SUBSECÇÃO II - ÁREAS DE ATIVIDADES ECONÓMICAS DE TIPO II",
    "39-44": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO IV - ESPAÇOS VERDES DE FRENTE ATLÂNTICA E RIBEIRINHA",
    "45-48": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO V - ESPAÇOS URBANOS DE BAIXA DENSIDADE",
    "49-51": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO VI - ESPAÇOS DE USO ESPECIAL - SUBSECÇÃO I - ESPAÇOS DE USO ESPECIAL: EQUIPAMENTOS",
    "52-54": "TITULO III - USO DO SOLO - CAPÍTULO III - QUALIFICAÇÃO FUNCIONAL - SECÇÃO VI - ESPAÇOS DE USO ESPECIAL - SUBSECÇÃO II - ESPAÇOS DE USO ESPECIAL: INFRAESTRUTURAS",
    "55-56": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO I - DISPOSIÇÕES GERAIS",
    "57-58": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL",
    "59-59": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO I - ESTRUTURA ECOLÓGICA MUNICIPAL",
    "60-64": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO I - ESTRUTURA ECOLÓGICA MUNICIPAL - SUBSECÇÃO I - ÁREAS DE GÉNESE NATURAL",
    "65-68": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO I - ESTRUTURA ECOLÓGICA MUNICIPAL - SUBSECÇÃO II - ESPAÇOS VERDES FUNDAMENTAIS",
    "69-72": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO I - ESTRUTURA ECOLÓGICA MUNICIPAL - SUBSECÇÃO III - CORREDORES VERDES",
    "73-74": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO I - ESTRUTURA ECOLÓGICA MUNICIPAL - SUBSECÇÃO IV - CORREDOR ECOLÓGICO E SUB-REGIÕES HOMOGÉNEAS DO PROF-EDM",
    "75-75": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO II - ÁREAS SUJEITAS A RISCOS NATURAIS",
    "76-76": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO II - ÁREAS SUJEITAS A RISCOS NATURAIS - SUBSECÇÃO I - PROGRAMA DA ORLA COSTEIRA CAMINHA-ESPINHO (POC-CE)",
    "77-78": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO II - ÁREAS SUJEITAS A RISCOS NATURAIS - SUBSECÇÃO II - OUTROS RISCOS NATURAIS",
    "79-82": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO II - SISTEMA AMBIENTAL - SECÇÃO III - ZONAMENTO ACÚSTICO",
    "83-84": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL",
    "85-86": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO I - PATRIMÓNIO URBANÍSTICO E ARQUITETÓNICO - SUBSECÇÃO I - ÁREAS DE INTERESSE URBANÍSTICO E ARQUITETÓNICO",
    "87-88": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO I - PATRIMÓNIO URBANÍSTICO E ARQUITETÓNICO - SUBSECÇÃO II - NÚCLEOS E LUGARES",
    "89-92": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO I - PATRIMÓNIO URBANÍSTICO E ARQUITETÓNICO - SUBSECÇÃO III - CONJUNTOS E IMÓVEIS DE VALOR PATRIMONIAL",
    "93-95": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO I - PATRIMÓNIO URBANÍSTICO E ARQUITETÓNICO - SUBSECÇÃO IV - CONJUNTOS E IMÓVEIS DE VALOR PATRIMONIAL CLASSIFICADOS OU EM VIAS DE CLASSIFICAÇÃO",
    "96-97": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO II - PATRIMÓNIO ARQUEOLÓGICO",
    "98-99": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO III - PATRIMÓNIO NATURAL",
    "100-101": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO III - SISTEMA PATRIMONIAL - SECÇÃO IV - ESTABELECIMENTOS DE INTERESSE HISTÓRICO E CULTURAL OU SOCIAL LOCAL",
    "102-103": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO IV - SISTEMAS DE CIRCULAÇÃO E TRANSPORTES - SECÇÃO I - DISPOSIÇÕES GERAIS",
    "104-105": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO IV - SISTEMAS DE CIRCULAÇÃO E TRANSPORTES - SECÇÃO II - CANAIS FERROVIÁRIOS",
    "106-107": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO IV - SISTEMAS DE CIRCULAÇÃO E TRANSPORTES - SECÇÃO III - CANAIS RODOVIÁRIOS",
    "108-113": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO IV - SISTEMAS DE CIRCULAÇÃO E TRANSPORTES - SECÇÃO III - CANAIS RODOVIÁRIOS - SUBSECÇÃO I - REDE MUNICIPAL",
    "114-115": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO IV - SISTEMAS DE CIRCULAÇÃO E TRANSPORTES - SECÇÃO IV - INTERFACES DE PASSAGEIROS E MERCADORIAS",
    "116-122": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO IV - SISTEMAS DE CIRCULAÇÃO E TRANSPORTES - SECÇÃO V - ESTACIONAMENTO",
    "123-127": "TITULO IV - DOS SISTEMAS URBANOS - CAPITULO V - SISTEMAS DE INFRAESTRUTURAS",
    "128-130": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO I - DISPOSIÇÕES GERAIS",
    "131-135": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO II - REGIME ECONÓMICO-FINANCEIRO - SECÇÃO I - EDIFICABILIDADE",
    "136-140": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO II - REGIME ECONÓMICO-FINANCEIRO - SECÇÃO II - ENCARGOS URBANÍSTICOS",
    "141-146": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO II - REGIME ECONÓMICO-FINANCEIRO - SECÇÃO III - ZONAMENTO INCLUSIVO E INCENTIVOS",
    "147-150": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO II - REGIME ECONÓMICO-FINANCEIRO - SECÇÃO IV - OUTROS INSTRUMENTOS DO REGIME ECONÓMICO E FINANCEIRO",
    "151-157": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO III - EXECUÇÃO - SECÇÃO I - FORMAS DE EXECUÇÃO",
    "158-160": "TITULO V - PEREQUAÇÃO, FINANCIAMENTO E EXECUÇÃO - CAPÍTULO III - EXECUÇÃO - SECÇÃO II - UNIDADES OPERATIVAS DE PLANEAMENTO E GESTÃO",
    "161-164": "TITULO VI - DISPOSIÇÕES GERAIS"
}

In [22]:
def get_hierarchy(article_number):
    for key, hierarchy in article_hierarchy.items():
        start, end = map(int, key.split('-'))
        if start <= article_number <= end:
            return hierarchy
    return "Article not found"
    

In [23]:
enriched_articles = []
for article in chunks:
    string = article[:10]
    pattern = r"(\d+)"  # Regex to capture the number
    match = re.findall(pattern, string)
    article_number = int(match[0])

    hierarchy = get_hierarchy(article_number)

    enriched_article = hierarchy + '\n' + article
    enriched_articles.append(enriched_article)

In [24]:
chunker.save_chunks_to_file(enriched_articles, '../data/enriched_articles.txt')

Chunks saved to ../data/enriched_articles.txt


In [15]:
with open('../data/enriched_articles.txt', 'w') as f:
    for article in enriched_articles:
        f.write(article + '\n')

# Vectorizer

In [29]:
vectorizer = TextVectorizer(api_key=os.getenv('MISTRAL_API_KEY'))

In [35]:
# Free API very low rate - introduced sleep in the method to overcome this
embeddings = []

for chunk in tqdm(enriched_articles):
    embeddings.append(vectorizer.get_text_embedding(chunk))
    time.sleep(2)

100%|██████████| 166/166 [06:12<00:00,  2.24s/it]


In [37]:
vectorizer.save_embeddings_to_db(np.array(embeddings), '../data/artigos_embeddings.faiss')    

# Retriever

In [38]:
retriever = Retriever('../data/artigos_embeddings.faiss', vectorizer)

In [66]:
layers = ['Qualificação do solo funcional - Espaços centrais - Área de frente urbana contínua do tipo II', ' Qualificação do solo operativa - Espaço consolidado', 'Corredores verdes - Corredor verde complementar - Corredor Verde da Granja (Constituição / Regado)', 'Área de Aluvião', 'Sub-regiões homogéneas - Grande Porto', 'Zonamento acústico - Zona mista', 'Unidades territoriais perequativas - Área Central']

In [74]:
all_relevant_chunks = []

for layer in layers:
    relevant_chunks = retriever.retrieve(layer, enriched_articles, k=5)
    all_relevant_chunks.extend(relevant_chunks)
    time.sleep(2)

In [78]:
all_relevant_chunks = set(all_relevant_chunks)

In [79]:
sum([len(x.split()) for x in all_relevant_chunks])

5061

# Question answering

In [83]:
question = "O que é que se pode construir aqui e quais as restrições?"
context = '\n'.join(all_relevant_chunks)
model = "mistral-large-latest"
client = Mistral(api_key=os.getenv('MISTRAL_API_KEY'))

In [84]:
system_prompt = """
        You are an expert document analyst with a focus on legal documents from public municipalities called Plano Director Municipal.
        
        I will provide you with:
        1 - the classification of the location according to the Plano Director Municipal
        2 - a series of relevant articles from the Plano Director Municipal that describe the details about the classification above
        3 - a question about that particular location that I need an answer to, based on its location classification and the articles provided
        
        You answer only to pertinent questions.
        For every answer, provide the article numbers ("Artigo"), sections ("SECÇÃO") or subsections ("SUBSECÇÃO") that support your answer and a detailed explanation.
        Take into consideration that for legal purposes, the most restrictive articles will take precedence over the less restrictive ones.
        You answer only in European Portuguese (PT-PT).
        """

In [85]:
layers_formatted = '\n'.join(layers)

In [86]:
question_prompt = f"""
In the Plano Director Municipal, the location is classified as:

{layers_formatted}

The relevant texts for the classification of this location are below:

---------------------
{all_relevant_chunks}
---------------------

Given the the location classification and the texts from the Plano Director Municipal that are provided, answer the question.

Question: {question}
Answer:
"""

In [90]:
print(question_prompt)


In the Plano Director Municipal, the location is classified as:

Qualificação do solo funcional - Espaços centrais - Área de frente urbana contínua do tipo II
 Qualificação do solo operativa - Espaço consolidado
Corredores verdes - Corredor verde complementar - Corredor Verde da Granja (Constituição / Regado)
Área de Aluvião
Sub-regiões homogéneas - Grande Porto
Zonamento acústico - Zona mista
Unidades territoriais perequativas - Área Central

The relevant texts for the classification of this location are below:

---------------------
{'TITULO I - DISPOSIÇÕES GERAIS\nArtigo 1.º\nÂmbito e Objetivos\n1 — O presente Regulamento, a Planta de Ordenamento e a Planta de Condicionantes são \npartes integrantes do Plano Diretor Municipal do Porto, adiante designado por PDMP ou Plano, \nelaborado ao abrigo do Regime Jurídico dos Instrumentos de Gestão Territorial (RJIGT) em vigor, \no qual estabelece as regras e orientações a que devem obedecer as ações de ocupação, o uso e a \ntransformação do

In [87]:
messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": question_prompt},
        ]
chat_response = client.chat.complete(
            model=model, messages=messages, temperature=0
        )

In [88]:
print(chat_response.choices[0].message.content)

De acordo com a classificação da localização e os artigos fornecidos do Plano Diretor Municipal, a área em questão é uma "Área de frente urbana contínua do tipo II" em "Espaço consolidado", situada em um "Corredor Verde Complementar" e "Área de Aluvião", dentro da "Zona mista" de zonamento acústico. Aqui estão as restrições e possibilidades de construção com base nos artigos relevantes:

1. **Área de Frente Urbana Contínua do Tipo II**:
   - **Artigo 26.º**: As áreas de frente urbana contínua do tipo II são áreas estruturadas em quarteirão com edifícios localizados, predominantemente, à face dos arruamentos. Pretende-se a manutenção e reestruturação das malhas e a consolidação do tipo de relação do edificado com o espaço público existente, designadamente, a uniformidade da frente urbana.
   - **Artigo 28.º**: Os logradouros e interior dos quarteirões devem ser permeáveis, sendo ocupados por coberto vegetal. Admite-se a criação de espaços de circulação, estadia, e/ou um anexo com o máxi